In [5]:
import numpy as np
import mne
import pandas as pd
import sys

import csv
from scipy.stats import skew, kurtosis
import pyeeg as p
from numpy import nan
import math


In [6]:
def all_psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 

    # Get only in postive frequencies
    fft_vals = np.absolute(np.fft.rfft(data))

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];


    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    psd_vals_list = []
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = psd_vals[:,freq_ix]
        for i in range(len(eeg_band_fft[band])): 
            psd_vals_list.append(np.mean(eeg_band_fft[band][i]) * 1000000)
    return psd_vals_list

In [13]:
seizure_var=0
def add_row(ip_psd, start, end):
    row_to_add = []
    
    ip_psd = ip_psd[:,start:end]
    ip_psd = ip_psd[:][0]
    
    
    p = all_psd(ip_psd)
    p.append(seizure_var)
    
    return p

In [8]:
data = mne.io.read_raw_edf('chb01_03.edf')
header = ','.join(data.ch_names)
df = pd.DataFrame(data[:][0])
df = df.transpose()

Extracting EDF parameters from F:\PY\Pro\pro\specto\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-8-e0b81f09e547>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_03.edf')


In [14]:
start = temp = 1996*256 
duration = 10*256
end = 2996*256

df1 = df.iloc[start:end, :]
df1 = pd.DataFrame(df1)
df1.shape

(256000, 23)

In [15]:

res = pd.DataFrame()

#first iteration run in 'w' mode, all subsequent iteration run in 'a' mode
#with open('test.csv', 'w') as file:
#    writer = csv.writer(file)
while temp < end:    
    row = add_row(data, temp, temp + duration)
    res=res.append(pd.Series(row),ignore_index=True)
#        writer.writerow(row)
    temp += duration

#res.columns = ['Variance', 'Kurtosis', 'Skewness', 'Petrosian Fractal Dimension', 'Hjorth Mobility', 'Hjorth Complexity', 'Spectral Entropy', 'Label']
res

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,178.786422,187.299138,373.456785,147.897919,433.100880,439.940230,333.774992,416.115508,394.691618,368.760198,...,0.410709,3.580521,0.574664,0.676765,0.389804,3.855179,8.851754,0.729672,0.410709,0.0
1,246.545195,146.974897,407.334715,258.835636,733.011971,259.251316,281.086734,447.975677,726.113512,372.659878,...,0.408117,3.408203,0.704777,0.795886,0.492413,4.095119,8.313062,0.824171,0.408117,0.0
2,154.673070,152.911484,346.467292,296.324842,448.180389,407.671629,272.390824,534.975166,525.624321,372.344138,...,0.321299,3.311277,0.696812,0.748675,0.366602,3.904908,8.709865,0.767086,0.321299,0.0
3,142.616877,123.756485,274.717083,262.639245,786.032023,453.893509,325.054145,585.962765,691.101114,508.397870,...,0.395649,3.612864,0.795568,0.716539,0.344873,3.920525,8.164512,0.807174,0.395649,0.0
4,260.544638,196.312737,576.709094,318.495857,711.977651,452.092105,368.535053,774.207210,544.819770,521.200692,...,0.350373,3.773968,0.753706,0.902003,0.380239,4.014634,7.777630,0.833583,0.350373,0.0
5,234.320907,261.550282,470.721632,286.447461,751.604378,469.626384,357.002351,577.517453,785.321363,489.468379,...,0.346077,3.542306,0.545660,0.555900,0.331486,3.823273,8.410936,0.756244,0.346077,0.0
6,273.379424,259.283579,521.113884,279.343219,571.597933,461.339238,522.689588,474.163002,568.110176,406.117779,...,0.352900,3.593283,0.635650,0.667032,0.392782,3.953978,8.084922,0.754650,0.352900,0.0
7,312.157337,327.162636,605.796649,393.135495,1041.770233,647.134773,511.948216,713.981772,1216.144852,716.313123,...,0.339203,3.759761,0.659787,0.565934,0.356684,3.929324,8.775190,0.805310,0.339203,0.0
8,90.867944,183.301948,179.605055,206.120312,340.883022,351.363763,279.098579,504.991956,267.143485,271.414462,...,0.363857,3.533879,0.541001,0.655530,0.331713,3.923096,8.709387,0.794712,0.363857,0.0
9,121.823402,143.578508,188.337742,315.325243,290.538303,463.355738,185.048609,490.961345,216.813588,398.905451,...,0.355697,3.714707,0.488094,0.660672,0.367932,4.054801,8.720214,0.790561,0.355697,0.0


In [11]:
df_final = pd.DataFrame()

In [16]:
df_final = df_final.append(res)

In [18]:
df_final.shape

(104, 116)

In [26]:
X = df_final.iloc[:,0:115]
y = df_final.iloc[:,-1]

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
rf_cs = RandomForestClassifier(n_jobs=-1,random_state=0)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [35]:
param_grid = { 'n_estimators': [10,50,100,200,250,500,700],}
    
CV_rfc = GridSearchCV(estimator=rf_cs, param_grid=param_grid, cv= 10)
CV_rfc.fit(X, y)
print (CV_rfc.best_params_)
op_RF = CV_rfc.best_estimator_
op_RF.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


{'n_estimators': 10}


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

AttributeError: 'RandomForestClassifier' object has no attribute 'test_score_'